In [1]:
import torch

C:\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Python39\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Python39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
## Initialize policy to be evaluated
class policy():
    def __init__(self):
        self.q = 0
        self.states = [0]
        self.actions = [0]
        
    def predict(state, action):
        q = state * action
        return q

In [ ]:
## Initialize value function arbitrarily
